# KNN on Mnist Dataset from scratch



Importing Library

In [ ]:
import numpy as np
from collections import Counter
import csv
import operator as op
import time

# Assignment Checkpoint 1

Loading Mnist Dataset:
    1. Reading csv file
    2. While returning dataset, performing some basic manpulation as follows
        a. training and test set first received with addition column of labels
        b. seperated training and testing images from labels and returned images and labels

In [ ]:
def load_mnist():
    
    with open("mnist_train.csv") as fl:
        rd = csv.reader(fl)
        trn_im = list(rd)
        trn_ = []
        for i in trn_im:
            i = list(map(int, i))
            trn_.append(i)

    with open("mnist_test.csv") as fl:
        rd = csv.reader(fl)
        tst_im = list(rd)
        tst_=[]
        for j in tst_im:
            j = list(map(int, j))
            tst_.append(j)


    return np.array(trn_)[:,1:np.shape(trn_)[1]], np.array(tst_)[:,1:np.shape(tst_)[1]],np.array(trn_)[:,0], np.array(tst_)[:,0]


Let's get started : Loading all data set

In [ ]:
train, test,train_label,test_label = load_mnist()


Calculating Euclidean Distance:
    1. Formula for euclidean distance is sqrt(sum(square(a-b)))
    2. The above formula can also be written as sqrt(sum(-2*ab+square(a)+square(b)))
    3. Using matrices manuplations for calculating euclidean distance of all testing images with training images


In [ ]:
def calculate_euclidean_distance(train_x, test_x):
    aDotb = np.dot(test_x, train_x.T)
    aSquare = np.square(test_x).sum(axis = 1)
    bSquare = np.square(train_x).sum(axis = 1)
    distance = np.sqrt(-2*aDotb + bSquare + np.matrix(aSquare).T)
    return distance

Finding indexes of k nearest neighbors using distances we got from calculate_euclidean_distance function

In [ ]:
def get_neighbor(distance,k):
    setSize = np.shape(distance)[0]
    
    neighbors = np.zeros((setSize,k))
    for j in range(0,setSize):
        neighbors[j] = np.argsort(distance[j])[:,0:k]
    return neighbors.astype(int)

From get_neighbor function we got indices of all the neighbors. Now below function will predict labels using those indices

In [ ]:
def predict(neighbors,train_y):
    labels = train_y[neighbors]
    predictions = []
    for i in range(0,np.shape(labels)[0]):
        sd, cv = np.unique(labels[i,:], return_counts = True)
        dc = dict(zip(sd,cv))
        z  = max(dc.items(), key = op.itemgetter(1))[0]
        predictions.append(z)

    return predictions

Calculating accuracy from our predictions

In [ ]:
def accuracy_score(predictions,actual):
    return (sum(predictions==actual)/len(predictions))*100
    

# Assignment Checkpoint 2

Program the k-Nearest Neighbor classifier algorithm with Euclidean distance for k=1 in Python

Accuracy : 96.91

In [ ]:
start = time.time()
k = 1

euclideanDistance =calculate_euclidean_distance(train, test)
neighbors=get_neighbor(euclideanDistance,k)
predictions = predict(neighbors,train_label)

print(accuracy_score(predictions,test_label))

end=time.time()
print("Excecution time for k as 1 is ",end-start, "seconds")


# Assignment Checkpoint 3 and 4

Apply 10-fold cross validation to find the optimal k (assuming k ranges from 1-10) for k-NN algorithm with Euclidean distance Note that these 10 folds should be selected from the training set.

In [ ]:
start = time.time()

setSize=(int)(train.shape[0]/10)
acc={1:[],2:[],3:[],4:[],5:[],6:[],7:[],8:[],9:[],10:[]}
pred=[]
for i in range(0,10):
    
    val_set=train[i*setSize:(i+1)*setSize,:]
    val_label=train_label[i*setSize:(i+1)*setSize,]
    train_set=np.delete(train,slice(i*setSize,(i+1)*setSize),0)
    train_lbl=np.delete(train_label,slice(i*setSize,(i+1)*setSize,))
  
    euclideanDistance =calculate_euclidean_distance(train_set, val_set)
    for k in range(1,11):
        neighbors=get_neighbor(euclideanDistance,k)
        predictions = predict(neighbors,train_lbl)
        acc[k].append(accuracy_score(predictions,val_label))
        pred

averageAcc=[]
for j in range(1,11):
    averageAcc.append(np.sum(acc[j])/10)
    print("Average accuracy when k was ",j,"is ",np.sum(acc[j])/10)             

maxIndex=np.argmax(averageAcc)




print("_________________________\n")
print("Optimal K is ", maxIndex+1," with accuracy as ",averageAcc[maxIndex]) #maxIndex +1 is because list starts from 0

optimalK=maxIndex+1
accuracy_kOptimal=averageAcc[maxIndex]


"the code you want to test stays here"
end = time.time()
print("Cross Validation execution time is ",end-start)

Now Running for Optimal K

In [ ]:
euclideanDistance =calculate_euclidean_distance(train, test)
neighbors=get_neighbor(euclideanDistance,optimalK)
predictions = predict(neighbors,train_label)

# Assignment Checkpoint 5

Confusion matrix and classification accuracy with confidence interval

In [ ]:
def confusion_matrix(y_true,y_pred):

    actual = np.array(y_true)
    predicted = np.array(y_pred)
    cm = np.zeros((10, 10))
    for a, p in zip(actual, predicted):
        cm[a][p] += 1

    return cm

In [ ]:
def printMatrix(s):

    print("     ", end="")
    for j in range(len(s[0])):
        
        print("%5d " % j, end="")
    print()
    print("     ", end="")
    for j in range(len(s[0])):
        print("------", end="")
    print()
    
    for i in range(len(s)):
        
        print("%3d |" % (i), end="") 
        for j in range(len(s[0])):
            print("%5d " % (s[i][j]), end="")
        print()


In [ ]:
printMatrix(confusion_matrix(test_label,predictions))

In [ ]:
def calculate_confidence_interval(accuracy):
    p=accuracy/100
    
    variance = np.sqrt((p*(1-p))/10000)
    cminus=p-(1.96*variance)
    cplus=p+(1.96*variance)
    return (cminus,cplus)

In [ ]:
print("Confidence Interval for optimal k is ",calculate_confidence_interval(accuracy_kOptimal))

In [ ]:
def pad_with(vector, pad_width, iaxis, kwargs):
    pad_value = kwargs.get('padder', 0)
    vector[:pad_width[0]] = pad_value
    vector[-pad_width[1]:] = pad_value
    return vector


# Assignment Checkpoint 6 and 7

Sliding Window, Confusion Matrix, Confidence Interval

In [ ]:
eucl= np.zeros((test.shape[0],train.shape[0]))
for i in range(train.shape[0]):
    t=train[i]
    ret=np.pad(np.reshape(t,(28,28)),1,pad_with)
    
    tempt=np.zeros((9,train.shape[1]))
    tempt[0]=t
    tempt[1]=np.reshape(ret[0:28,0:28],(1,28*28))
    tempt[2]=np.reshape(ret[1:29,0:28],(1,28*28))
    tempt[3]=np.reshape(ret[2:30,0:28],(1,28*28))
    tempt[4]=np.reshape(ret[0:28,1:29],(1,28*28))
    tempt[5]=np.reshape(ret[0:28,2:30],(1,28*28))
    tempt[6]=np.reshape(ret[2:30,2:30],(1,28*28))
    tempt[7]=np.reshape(ret[1:29,2:30],(1,28*28))
    tempt[8]=np.reshape(ret[2:30,1:29],(1,28*28))
    euclidean=calculate_euclidean_distance(tempt, test)

    
    for c in range(euclidean.shape[0]):
        
        min_value=np.min(euclidean[c][:])
        
        eucl[c,i]=min_value

        
        
    
    
    
neighbors=get_neighbor(np.matrix(eucl),optimalK)
predictions = predict(neighbors,train_label)
printMatrix(confusion_matrix(test_label,predictions))
accuracy_window=accuracy_score(predictions,test_label)
print("Confidence interval for window slide is ",calculate_confidence_interval(accuracy_window))
print("Accuracy of window sliding with Otimale k as 3 is ",accuracy_window)    
    

# Assignment Checkpoint 8

Siginificance Testing and Difference rule is explained in report

In [ ]:
def siginificance_testing(accuracy1,accuracy2):
    p1=accuracy1/100
    p2=accuracy2/100
    print("Null Hypothesis is that accuracy 1 == accuracy 2\n")
    print("Alternate Hypothesis is that accuracy 1 != accuracy2\n")

    diff= p2-p1

    if(diff==0):
        print("Null Hypothesis is true")
    else:
        a=(p2+p1)/2
        print(a)
        SD=np.sqrt((a*(1-a))/20000)
        print(SD)
        cminus=-1*1.96*SD
        cplus=1.96*SD
        if(diff>cplus):
            print("We are rejecting null hypothesis")
        else:
            if(diff<cminus):
                print("We are rejecting null hypothesis")
            else:
                print("We are accepting null hypothesis")
   
    
siginificance_testing(accuracy_kOptimal,accuracy_window)